In [3]:
import numpy as np
import pandas as pd

### Make Train Ref File

In [35]:
#Fetch the training data
train_data = pd.read_csv('data/train_dssm3.csv')
train_data.head(6)

,product_uid,product_title,search_term,relevance,brand,product_description
0,116711,ge z wave 1800 watt resist cfl led indoor plug...,zwave switch,3.00,NaN,transform ani home into a smart home with the ...
1,141628,leviton z wave control 3 way/remot scene capab...,zwave switch,3.00,leviton,the leviton dzmx1 is a z wave enabl univers di...
2,142033,leviton decora z wave control 15 amp scene cap...,zwave switch,3.00,leviton,the leviton dzs15 is a z wave enabl univers sw...
3,135547,ge z wave 600 watt cfl led indoor in wall dimm...,zwave switch,2.67,NaN,transform ani home into a smart home with the ...
4,152640,z wave wireless light control with keypad control,zwave switch,2.67,NaN,transform ani home into a smart home with the ...
5,113534,lutron aurora wireless light control system li...,zwave switch,2.33,NaN,aurora is a pre program wireless radio frequen...


In [36]:
splitVals = int(len(train_data)*0.80)
train_data[:splitVals].to_csv('data/train_final.csv',index=False)
train_data[splitVals:].to_csv('data/dev_final.csv',index=False)

In [37]:
master_dict_train = {}
for term in list(set(train_data['search_term'])):
    master_dict_train[term] = len(master_dict_train)+1

In [38]:
with open('data/reference/s_dict_train.txt','w') as f:
    for k,v in master_dict_train.items():
        f.write(k+'||'+str(v)+'\n')

In [39]:
dev_data = train_data[splitVals:]
gr = dev_data.groupby('search_term')['relevance'].apply(list).reset_index(name='relevance')
pid = dev_data.groupby('search_term')['product_uid'].apply(list).reset_index(name='product_uid')
merged = gr.merge(pid,how='left',on='search_term')

In [40]:
merged['search_term'] = merged['search_term'].apply(lambda x: str(master_dict_train[x]))

In [41]:
merged

,search_term,relevance,product_uid
0,6986,"[3.0, 2.67, 2.33]","[102253, 111283, 102836]"
1,3719,"[3.0, 3.0, 2.33]","[120307, 131403, 142574]"
2,8868,"[2.67, 2.33, 2.33, 2.0, 2.0, 2.0, 2.0, 2.0, 1.67]","[173029, 103075, 172086, 103297, 127687, 16977..."
3,1029,"[3.0, 2.67, 2.33, 2.33, 2.33, 2.33, 2.0, 1.33]","[152423, 133273, 109031, 142783, 143108, 16139..."
4,9026,"[2.67, 2.67, 2.67, 2.67, 2.33, 1.67]","[152672, 153135, 158820, 162809, 137444, 133249]"
...,...,...,...
2404,7020,"[3.0, 3.0, 3.0, 2.67, 2.67, 2.0]","[102455, 117151, 162801, 100342, 133562, 126083]"
2405,11441,"[3.0, 2.33]","[125934, 146854]"
2406,8649,"[3.0, 3.0, 2.67, 2.67, 2.67, 2.33]","[134539, 163251, 103879, 116416, 138406, 156518]"
2407,6518,"[3.0, 3.0, 2.67, 2.67, 2.33, 2.33]","[104260, 114319, 103819, 107799, 111408, 140684]"


In [42]:
# torch.argsort(torch.argsort(torch.tensor([5,7,3,1,9,11,4], dtype=torch.float32), descending=True))

In [43]:
merged_dev = merged.reset_index().drop(['index'],axis=1)

In [44]:
with open('data/reference/dev.out','w') as ft:
    for i in range(len(merged_dev)):
        argsorted = np.argsort(np.array(merged_dev['relevance'][i]))
        app_lst = []

        for j in range(len(argsorted)):
            app_lst.append( str(merged_dev['product_uid'][i][j] ) + '-' + str(argsorted[j]) )

        ft.write( merged_dev['search_term'][i] + ' ' + ",".join(app_lst) + '\n')

### Make Test Ref File

In [4]:
test_data = pd.read_csv('data/test_final.csv',encoding='ISO-8859-1').dropna().reset_index().drop(['index'],axis=1).sort_values(['search_term','relevance'], ascending=[True,False])
test_data.head(3)

,product_uid,product_title,search_term,relevance,brand,product_description
1024,711159484,top 16gb 8gb class10 6 microsd sdhc micro sd t...,16 gb memori card,4.0,<UNK>,<UNK>
1026,711159486,new sandisk 16gb class 4 microsd microsdhc sd ...,16 gb memori card,4.0,sandisk,irrespect of the size of your phone or other d...
1027,711159487,lot of 2 sandisk 16gb micro sd microsdhc sdhc ...,16 gb memori card,4.0,<UNK>,<UNK>


In [5]:
master_dict_test = {}
for term in list(set(test_data['search_term'])):
    master_dict_test[term] = len(master_dict_test)+1

In [6]:
with open('data/reference/s_dict_test.txt','w') as f:
    for k,v in master_dict_test.items():
        f.write(k+'||'+str(v)+'\n')

In [7]:
gr_test = test_data.groupby('search_term')['relevance'].apply(list).reset_index(name='relevance')
pid_test = test_data.groupby('search_term')['product_uid'].apply(list).reset_index(name='product_uid')
merged_test = gr_test.merge(pid_test,how='left',on='search_term')
merged_test['search_term'] = merged_test['search_term'].apply(lambda x: str(master_dict_test[x]))

In [11]:
with open('data/reference/test.out','w') as ft:
    for i in range(len(merged_test)):
#         argsorted = np.argsort(np.argsort(-np.array(merged_test['relevance'][i])))
        argsorted = [ i for i in range (len(merged_test['relevance'][i])) ]
                     
        app_lst = []

        for j in range(len(argsorted)):
            app_lst.append( str(merged_test['product_uid'][i][j] ) + '-' + str(argsorted[j]) )

        ft.write( merged_test['search_term'][i] + ' ' + ",".join(app_lst) + '\n')